In [10]:
import numpy as np
import pandas as pd

In [11]:
df = pd.read_csv("../../data/jx_duty.csv")
df.head(5)

,Unnamed: 0,X,shixin_id,duty
0,4,4,18095,被告李华于判决生效之日起三日内给付原告郭万清购车款人民币元案件受理费元 邮寄费元 合计元 由...
1,5,5,18196,一被告任晓平偿还原告罗秀花欠款元二本案诉讼费元 由被告任晓平自愿承担
2,6,6,18542,被告寇先强于本判决生效之日起十日内赔偿给原告共计元
3,7,7,21388,判决如下 一被告陈雪明向原告农四师农业生产资料公司六十四团分公司支付农业生产资料款元 利息元...
4,8,8,21456,被告苏宪礼自愿赔偿原告王世霜各项经济损失元整此款被告苏宪礼于年月日前支付元 于年月日前支付元...


In [12]:
df.shape

(50284, 4)

In [13]:
df = df.sample(20000)

In [14]:
import jieba

In [15]:
# Need to be length greater than 10;

def length_bigger_than_10(text):
    if len(text)>=10:
        return True
    else:
        return False


#新建length列
df['length'] = df.agg({'duty': length_bigger_than_10})
#根据length列来保留长度大于10的文本，并将其放入新的dataframe中
df = df[df['length']==True]

In [16]:
import nltk
import jieba

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leijunjie/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
#停止词
from nltk.corpus import stopwords
stopwords = stopwords.words("chinese")



def clean_text(text):
    wordlist = jieba.lcut(text) 
    #去除停用词和长度小于2的词语
    wordlist = [w for w in wordlist if w not in stopwords and len(w)>2]
    #将中文数据组织成类似西方于洋那样，词语之间以空格间隔
    document =  " ".join(wordlist)
    return document


#使用dataframe.agg对content列实行clean_text方法
df['duty'] = df.agg({'duty': clean_text})
corpus = df['duty']
corpus.head()

38802                         中国建设银行 塔城地区 刘艳周 戴姣凤马 马翠花 成马色
39319    王治彬 樊万平 医疗费 王治彬 樊万平 误工费 王治彬 樊万平 赔偿金 王治彬 樊万平 交通...
16502                                            热孜亚 古丽娜尔元
38946    财产保险 乌鲁木齐 支公司 财产保险 昌吉回族自治州 交强险 财产保险 乌鲁木齐 支公司 第...
10260                   钢结构 哈密市 物资贸易 钢结构 哈密市 物资贸易 哈密市 物资贸易
Name: duty, dtype: object

In [19]:
## jieba 分词包

def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [20]:
df["duty_cutted"] = df.duty.apply(chinese_word_cut)

In [21]:
df.duty_cutted.head()

38802               中国建设银行   塔城地区   刘艳周   戴姣凤马   马翠花   成马色
39319    王治彬   樊万平   医疗费   王治彬   樊万平   误工费   王治彬   樊万平 ...
16502                                          热孜亚   古丽娜尔元
38946    财产保险   乌鲁木齐   支公司   财产保险   昌吉回族自治州   交强险   财产保...
10260     钢结构   哈密市   物资贸易   钢结构   哈密市   物资贸易   哈密市   物资贸易
Name: duty_cutted, dtype: object

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [23]:
stpwrdpath = "../../data/stopwords.txt"
stpwrd_dic = open(stpwrdpath, 'rb')
stpwrd_content = stpwrd_dic.read().decode('utf-8')#将停用词表转换为list
stpwrdlst = stpwrd_content.splitlines()
stpwrd_dic.close()

In [24]:
n_features = 1000

In [25]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                #stop_words='english',
                                stop_words=stpwrdlst,
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.duty_cutted)

In [26]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                #stop_words='english',
                                stop_words=stpwrdlst,
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(corpus)

In [27]:
from sklearn.decomposition import LatentDirichletAllocation

In [28]:
lda = LatentDirichletAllocation(n_components = 15,max_iter=50,
                                learning_method='online',
                                learning_offset=50,
                                random_state=0)

In [29]:
lda.fit(tf)

LatentDirichletAllocation(learning_method='online', learning_offset=50,
                          max_iter=50, n_components=15, random_state=0)

In [30]:
import pyLDAvis 
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11     0.238838 -0.234110       1        1  9.739229
6      0.000956  0.078597       2        1  8.705551
7      0.037063  0.142691       3        1  8.478415
5      0.223668 -0.239276       4        1  8.171433
8     -0.286672 -0.187409       5        1  7.707416
4      0.018519  0.145000       6        1  6.793142
14     0.019586  0.094744       7        1  6.617519
9     -0.017297  0.047584       8        1  6.018053
2      0.007119  0.053774       9        1  5.950600
0     -0.265725 -0.172419      10        1  5.652849
1      0.000901  0.087621      11        1  5.479506
12     0.008284  0.079288      12        1  5.410874
10     0.000307  0.055334      13        1  5.225105
3      0.031142  0.010153      14        1  5.140504
13    -0.016689  0.038429      15        1  4.909805, topic_info=     Term         Freq        Total Category  logprob  loglift
5     一次性  2576.000000  2576.000000  Default  30.0000  30.0000
294   房地产  1592.000000  1592.000000  Default  29.0000  29.0000
154   十五日  1429.000000  1429.000000  Default  28.0000  28.0000
528  连带责任  1217.000000  1217.000000  Default  27.0000  27.0000
129  农村信用   956.000000   956.000000  Default  26.0000  26.0000
..    ...          ...          ...      ...      ...      ...
594   马国雄    25.156315    26.089173  Topic15  -4.7298   2.9775
178   叶城县    24.236868    25.169727  Topic15  -4.7670   2.9762
539  金晖兆丰    23.610647    24.543505  Topic15  -4.7932   2.9752
405   王志刚    23.494366    24.427224  Topic15  -4.7982   2.9750
430   第一款    53.468604    61.444506  Topic15  -3.9758   2.8749

[511 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1         5  0.754753    一三四团
1         7  0.215644    一三四团
2         1  0.157846     一个月
2         4  0.839078     一个月
3        13  0.958930     一二三
...     ...       ...     ...
601       3  0.954355     马金梅
602      15  0.995293    驳回上诉
603      14  0.984834     驾驶员
604       2  0.919255  高级人民法院
605      13  0.952704     魏昌会

[492 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 7, 8, 6, 9, 5, 15, 10, 3, 1, 2, 13, 11, 4, 14])

In [31]:
p = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
# pyLDAvis.save_html(p, 'lda_10_jx.html')

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
